### Bayes Classification of MNIST Dataset

In [ ]:
from __future__ import print_function, division
from future.utils import iteritems
from builtins import range, input
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from keras.datasets import mnist
from scipy.stats import norm
from scipy.stats import multivariate_normal as mvn

In [ ]:
from numba import jit

class Bayes(object):
    """A class for Bayes classifier.

    Computes the probability as:  P(Y|X) = P(X|Y) * P(Y) / P(X)
    or, posterior = likelihood * prior / evidence
    """

    def fit(self, X, Y, smoothing=1e-2):
        N, D = X.shape
        self.gaussian = dict()
        self.priors = dict()
        labels = set(Y)
        for c in labels:
            current_x = X[Y == c]
            self.gaussian[c] = {
                "mean": current_x.mean(axis=0),
                "cov": np.cov(current_x.T) + np.eye(D) * smoothing,
            }
            self.priors[c] = float(len(Y[Y == c])) / len(Y)

    def score(self, X, Y):
        P = self.predict(X)
        return np.mean(P == Y)

    # Main Bayes Classifier
    def predict(self, X):
        N, D = X.shape
        K = len(self.gaussian)
        log_P = np.zeros((N, K))
        for c, g in self.gaussian.items():
            mean, cov = g["mean"], g["cov"]
            cov += np.eye(D) * 1e-6  # regularize covariance
            cov_inv = np.linalg.inv(cov)
            det_cov = np.linalg.det(cov)
            log_prior = np.log(self.priors[c])
            for i in range(N):
                x = X[i]
                diff = x - mean
                log_exponent = -0.5 * diff.dot(cov_inv).dot(diff)
                log_likelihood = (
                    log_exponent
                    - 0.5 * D * np.log(2 * np.pi)
                    - 0.5 * np.log(np.abs(det_cov) + 1e-6)
                )
                log_P[i, c] = log_likelihood + log_prior

        return np.argmax(log_P, axis=1)

Prepare the MNIST Dataset

In [ ]:
# Load the data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Flatten the data
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Normalize the pixel values to be between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

Fit a Naive Bayes Classifier

In [ ]:
model = Bayes()
model.fit(X_train, Y_train)

Classify the test data

In [ ]:
print("Train accuracy:", model.score(X_train, Y_train))
print("Test accuracy:", model.score(X_test, Y_test))

Mean of each class

In [ ]:
for c, g in model.gaussian.items():
    plt.imshow(g["mean"].reshape(28, 28))
    plt.title(c)
    plt.show()